In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/application_train.csv


In [2]:
train = pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')
test = pd.read_csv('/kaggle/input/home-credit-default-risk/application_test.csv')
train.shape, test.shape

((307511, 122), (48744, 121))

In [3]:
missing_cols = [(col, train[col].isna().sum()) for col in train.columns if train[col].isna().any()]
missing_cols.extend([(col, test[col].isna().sum()) for col in test.columns if test[col].isna().any()])
missing_cols

[('AMT_ANNUITY', 12),
 ('AMT_GOODS_PRICE', 278),
 ('NAME_TYPE_SUITE', 1292),
 ('OWN_CAR_AGE', 202929),
 ('OCCUPATION_TYPE', 96391),
 ('CNT_FAM_MEMBERS', 2),
 ('EXT_SOURCE_1', 173378),
 ('EXT_SOURCE_2', 660),
 ('EXT_SOURCE_3', 60965),
 ('APARTMENTS_AVG', 156061),
 ('BASEMENTAREA_AVG', 179943),
 ('YEARS_BEGINEXPLUATATION_AVG', 150007),
 ('YEARS_BUILD_AVG', 204488),
 ('COMMONAREA_AVG', 214865),
 ('ELEVATORS_AVG', 163891),
 ('ENTRANCES_AVG', 154828),
 ('FLOORSMAX_AVG', 153020),
 ('FLOORSMIN_AVG', 208642),
 ('LANDAREA_AVG', 182590),
 ('LIVINGAPARTMENTS_AVG', 210199),
 ('LIVINGAREA_AVG', 154350),
 ('NONLIVINGAPARTMENTS_AVG', 213514),
 ('NONLIVINGAREA_AVG', 169682),
 ('APARTMENTS_MODE', 156061),
 ('BASEMENTAREA_MODE', 179943),
 ('YEARS_BEGINEXPLUATATION_MODE', 150007),
 ('YEARS_BUILD_MODE', 204488),
 ('COMMONAREA_MODE', 214865),
 ('ELEVATORS_MODE', 163891),
 ('ENTRANCES_MODE', 154828),
 ('FLOORSMAX_MODE', 153020),
 ('FLOORSMIN_MODE', 208642),
 ('LANDAREA_MODE', 182590),
 ('LIVINGAPARTMENTS_MO

In [4]:
train_copy = train.copy()
test_copy = test.copy()

train_y = train_copy['TARGET']
train_x = train_copy.drop('TARGET', axis=1)
test_x = test_copy

train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)

train_x, test_x = train_x.align(test_x, join = 'inner', axis = 1)
train_x.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,...,0,0,0,0,0,0,1,0,1,0
1,100003,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,...,0,1,0,0,0,0,0,0,1,0
2,100004,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,...,0,0,0,0,0,0,0,0,0,0
4,100007,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
for col in train_x.columns:
    print(train_x[col].describe())

count    307511.000000
mean     278180.518577
std      102790.175348
min      100002.000000
25%      189145.500000
50%      278202.000000
75%      367142.500000
max      456255.000000
Name: SK_ID_CURR, dtype: float64
count    307511.000000
mean          0.417052
std           0.722121
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max          19.000000
Name: CNT_CHILDREN, dtype: float64
count    3.075110e+05
mean     1.687979e+05
std      2.371231e+05
min      2.565000e+04
25%      1.125000e+05
50%      1.471500e+05
75%      2.025000e+05
max      1.170000e+08
Name: AMT_INCOME_TOTAL, dtype: float64
count    3.075110e+05
mean     5.990260e+05
std      4.024908e+05
min      4.500000e+04
25%      2.700000e+05
50%      5.135310e+05
75%      8.086500e+05
max      4.050000e+06
Name: AMT_CREDIT, dtype: float64
count    307499.000000
mean      27108.573909
std       14493.737315
min        1615.500000
25%       16524.000000
50%       24903.000000
75

count    157504.000000
mean          0.977065
std           0.064575
min           0.000000
25%           0.976700
50%           0.981600
75%           0.986600
max           1.000000
Name: YEARS_BEGINEXPLUATATION_MODE, dtype: float64
count    103023.000000
mean          0.759637
std           0.110111
min           0.000000
25%           0.699400
50%           0.764800
75%           0.823600
max           1.000000
Name: YEARS_BUILD_MODE, dtype: float64
count    92646.000000
mean         0.042553
std          0.074445
min          0.000000
25%          0.007200
50%          0.019000
75%          0.049000
max          1.000000
Name: COMMONAREA_MODE, dtype: float64
count    143620.000000
mean          0.074490
std           0.132256
min           0.000000
25%           0.000000
50%           0.000000
75%           0.120800
max           1.000000
Name: ELEVATORS_MODE, dtype: float64
count    152683.000000
mean          0.145193
std           0.100977
min           0.000000
25%           0

count    307511.000000
mean          0.008130
std           0.089798
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: FLAG_DOCUMENT_18, dtype: float64
count    307511.000000
mean          0.000595
std           0.024387
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: FLAG_DOCUMENT_19, dtype: float64
count    307511.000000
mean          0.000507
std           0.022518
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: FLAG_DOCUMENT_20, dtype: float64
count    307511.000000
mean          0.000335
std           0.018299
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: FLAG_DOCUMENT_21, dtype: float64
count    265992.000000
mean          0.006402
std           0.083849
min           0.000000
25%           0.

count    307511.000000
mean          0.060495
std           0.238403
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: OCCUPATION_TYPE_Drivers, dtype: float64
count    307511.000000
mean          0.001831
std           0.042749
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: OCCUPATION_TYPE_HR staff, dtype: float64
count    307511.000000
mean          0.037007
std           0.188779
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: OCCUPATION_TYPE_High skill tech staff, dtype: float64
count    307511.000000
mean          0.001711
std           0.041323
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: OCCUPATION_TYPE_IT staff, dtype: float64
count    307511.000000
mean          0.179460
std           0.38

count    307511.000000
mean          0.001288
std           0.035862
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: ORGANIZATION_TYPE_Realtor, dtype: float64
count    307511.000000
mean          0.000276
std           0.016623
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: ORGANIZATION_TYPE_Religion, dtype: float64
count    307511.000000
mean          0.005889
std           0.076515
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: ORGANIZATION_TYPE_Restaurant, dtype: float64
count    307511.000000
mean          0.028919
std           0.167580
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: ORGANIZATION_TYPE_School, dtype: float64
count    307511.000000
mean          0.010559
std           0.102213


In [6]:
specious_cols = []
for col in train_x.columns:
    if train_x[col].std() > 10:
        specious_cols.append(col)
specious_cols

['SK_ID_CURR',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'DAYS_LAST_PHONE_CHANGE']

In [7]:
train_x['DAYS_EMPLOYED'] = train_x['DAYS_EMPLOYED'].mask(np.isclose(train_x['DAYS_EMPLOYED'], 365243), np.nan)
test_x['DAYS_EMPLOYED'] = test_x['DAYS_EMPLOYED'].mask(np.isclose(test_x['DAYS_EMPLOYED'], 365243), np.nan)
train_x['DAYS_EMPLOYED'].describe()

count    252137.000000
mean      -2384.169325
std        2338.360162
min      -17912.000000
25%       -3175.000000
50%       -1648.000000
75%        -767.000000
max           0.000000
Name: DAYS_EMPLOYED, dtype: float64

In [8]:
bureau = pd.read_csv('/kaggle/input/home-credit-default-risk/bureau.csv')
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [9]:

atm_previous_loan = bureau[bureau['CREDIT_ACTIVE'] == 'Active'].groupby('SK_ID_CURR', as_index=False)['AMT_CREDIT_SUM'].sum()
print(atm_previous_loan.head() )

previous_active_overdue = bureau[bureau['CREDIT_ACTIVE'] == 'Active'].groupby('SK_ID_CURR', as_index=False)['CREDIT_DAY_OVERDUE'].sum().rename(columns = {'CREDIT_DAY_OVERDUE': 'previous_active_overdue'})
print(previous_active_overdue.describe() )

previous_closed_overdue = bureau[bureau['CREDIT_ACTIVE'] == 'Closed'].groupby('SK_ID_CURR', as_index=False)['CREDIT_DAY_OVERDUE'].sum().rename(columns = {'CREDIT_DAY_OVERDUE': 'previous_closed_overdue'})
print(previous_closed_overdue.describe() )


   SK_ID_CURR  AMT_CREDIT_SUM
0      100001      884025.000
1      100002      481988.565
2      100003      810000.000
3      100005      598626.000
4      100008      267606.000
          SK_ID_CURR  previous_active_overdue
count  251815.000000            251815.000000
mean   278208.986832                 4.533407
std    102862.499983                86.807593
min    100001.000000                 0.000000
25%    189065.000000                 0.000000
50%    278126.000000                 0.000000
75%    367258.000000                 0.000000
max    456255.000000              5250.000000
          SK_ID_CURR  previous_closed_overdue
count  267925.000000            267925.000000
mean   278049.104320                 0.421681
std    102823.410284                25.988253
min    100001.000000                 0.000000
25%    188847.000000                 0.000000
50%    277972.000000                 0.000000
75%    367151.000000                 0.000000
max    456255.000000              2347

In [10]:
from functools import reduce
train_x = reduce(lambda left,right: pd.merge(left,right,on = 'SK_ID_CURR', how = 'left'), [train_x, atm_previous_loan, previous_active_overdue,previous_closed_overdue])

test_x = reduce(lambda left,right: pd.merge(left,right,on = 'SK_ID_CURR', how = 'left'), [test_x, atm_previous_loan, previous_active_overdue,previous_closed_overdue])
train_x.shape, test_x.shape

((307511, 245), (48744, 245))

In [11]:
# correlations = train_x.corrwith(train_y,axis=1).sort_values()

# print('Positive Correlations:\n', correlations.tail(15))
# print('Negative Correlations:\n', correlations.head(15))
# f, axes = plt.subplots(2, 2, figsize=(15,10), sharex=False)
# sns.distplot(train_x['DAYS_BIRTH'], ax=axes[0, 0])
# sns.distplot(train_x['DAYS_EMPLOYED'], ax=axes[0, 1])
# sns.distplot(train_x['DAYS_ID_PUBLISH'], ax=axes[1, 0])
# plt.tight_layout()

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
# # model = RandomForestClassifier(n_estimators=100, max_depth=2)
import xgboost as xgb

train_X, valid_X, train_Y, valid_Y = train_test_split(train_x, train_y, test_size=0.2)

model = xgb.XGBClassifier()
model.fit(train_X, train_Y)
train_pred = model.predict_proba(train_X)[:, 1]
valid_pred = model.predict_proba(valid_X)[:, 1]
train_auc = roc_auc_score(train_Y, train_pred)
valid_auc = roc_auc_score(valid_Y, valid_pred)
print(train_auc, valid_auc)




In [ ]:
model.get_params, model.feature_importances_



In [ ]:
test_y = model.predict_proba(test_x)[:,1]
test_y[0]

In [ ]:
submission = pd.DataFrame({'SK_ID_CURR': test_x['SK_ID_CURR'], 'TARGET': test_y})
submission.head()
submission.to_csv('output.csv', index = False)